In [1]:
!pip install efficient-apriori

In [2]:
# i = 8
# j = 9
# i + j

In [3]:
import pandas as pd
# import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import numpy as np

from sklearn.model_selection import GroupKFold
from surprise.model_selection import PredefinedKFold
# from scipy.stats import permutation_test
from sklearn.model_selection import train_test_split
from surprise import Dataset, SVD
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import cohen_kappa_score, mean_absolute_error , mean_squared_error
from efficient_apriori import apriori
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
ratings = pd.read_csv("/kaggle/input/bookrecommender/BX-Book-Ratings.csv",encoding='ISO-8859–1',delimiter=";")
# books = pd.read_csv("BX-Books_small.csv",encoding='ISO-8859–1',delimiter=";")
# users = pd.read_csv("/kaggle/input/bookrecommender/BX-Users.csv",encoding='ISO-8859–1',delimiter=";")

In [5]:
ratings = ratings.rename(columns={"User-ID":"userID",
                                  "ISBN":"itemID",
                                  "Book-Rating":"rating"})
ratings['userID'] = ratings['userID'].astype(str)
ratings['itemID'] = ratings['itemID'].astype(str)
ratings['rating'] = ratings['rating'].astype(int)

ratings_grp = ratings.groupby(by="itemID",as_index=False).count()
ratings_grp = ratings_grp.loc[ratings_grp.userID > 100, :]
ratings = ratings.loc[ratings.itemID.isin(ratings_grp['itemID']),:]

ratings_grp2 = ratings.groupby(by="userID",as_index=False).count()
ratings_grp2 = ratings_grp2.loc[ratings_grp2.itemID > 10, :]
ratings = ratings.loc[ratings.userID.isin(ratings_grp2['userID']),:]
len(ratings)

4390

In [6]:
ratings_grp = ratings.groupby(by="userID",as_index=False)["itemID"].agg(list)
ratings_grp

,userID,itemID
0,10314,"[0060934417, 0060976845, 0061009059, 014200020..."
1,10447,"[0060928336, 0060934417, 0060976845, 006101351..."
2,11601,"[0060938455, 0312278586, 0316569321, 031660195..."
3,11676,"[0060392452, 0060502258, 0060928336, 006093053..."
4,11993,"[0060928336, 0142001740, 0312291639, 044020615..."
...,...,...
207,95359,"[0060392452, 0060502258, 0060928336, 006093441..."
208,95932,"[0060502258, 0060928336, 0060930535, 006098710..."
209,96054,"[0060502258, 0061009059, 0142001740, 015602732..."
210,9747,"[0060976845, 0140293248, 0312195516, 031227858..."


In [7]:
itemsets, rules = apriori(ratings_grp["itemID"].tolist(), min_support=0.05, min_confidence=0.8)

# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule)  # Prints the rule and its confidence, support, lift, ...

{0060928336, 0060987103} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0060976845, 0671021001} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0316096199, 067976402X} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0345370775, 0684872153} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0385484518, 0679781587} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0671021001, 0679781587} -> {0316666343} (conf: 0.800, supp: 0.075, lift: 1.631, conv: 2.547)
{0060502258, 0060930535} -> {0316666343} (conf: 0.800, supp: 0.057, lift: 1.631, conv: 2.547)
{0060502258, 0679781587} -> {0316666343} (conf: 0.800, supp: 0.057, lift: 1.631, conv: 2.547)
{0060502258, 0804106304} -> {0316666343} (conf: 0.800, supp: 0.057, lift: 1.631, conv: 2.547)
{0060934417, 0385484518} -> {0316666343} (conf: 0.800, supp: 0.057, lift: 1.631, conv: 2.547)
{0060976845, 0345443284} -> {0316666343} (conf: 0.800, supp:

In [8]:
# rules

In [9]:
# association_rules = apriori(ratings_grp["itemID"].tolist(), min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
# association_results = list(association_rules)

In [10]:
with open('association_rules.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump({"itemsets":itemsets,"rules":rules}, file)